In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

In [ ]:
from datetime import datetime
import time

## 동적 크롤링 위해 필요한 모듈 
from selenium import webdriver # --> pip install selenium 통해 직접 설치

# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


## checkRunningTime은 함수의 실행시간을 측정해준다.
def checkRunningTime(func):
    
    def new_func(*args, **kwargs):
        
        ## 시작시각 저장하기
        start_time = time.time()
        
        ## 함수 실행
        func(*args, **kwargs)
        
        ## 종료시각 저장하기
        end_time = time.time()
        
        print(f"<< 실행시간은 : {end_time - start_time:.3f}초 >>\n\n")
        
    return new_func

## Weather class에서 필요한 함수들을 static method을 정의해놓은 것을 모아놓은 클래스이다.
class UTIL:
    
    ## 정수형 입력 여부 체크를 위한 함수
    @staticmethod
    def isNum(string):

        ## for loop로 각각의 char를 확인하며 아스키코드상 "0"과 "9" 사이에 있는지 체크한다.
        for letter in string:
            if not ("0" <= letter <= "9"):
                return False

        return True

    ## 유효한 년도를 입력했는지 여부 판별 함수
    @staticmethod
    def is_valid_year(year):

        ## 정수가 아니라면 유효하지 않다.
        if not UTIL.isNum(year):
            return False

        ## year은 1960이상의 정수만 유효하다. 기상청 날씨누리는 1960년 이후의 정보만 있기 때문이다.
        if 1960 <= int(year):
            return True

        else:
            return False

    ## 유효한 달을 입력했는지 여부 판별 함수
    @staticmethod
    def is_valid_month(month):

        ## 정수가 아니라면 유효하지 않다.
        if not UTIL.isNum(month):
            return False

        ## month는 1이상 12이하만 유효하다.
        if 1<= int(month) <= 12:
            return True

        else:
            return False


    ## 유효한 일을 입력했는지 여부 판별 함수
    ## year과 month에 따라서 같은 day가 유효할 수도, 아닐수도 있기때문에 파라미터로 셋다 집어넣어주어야 한다.
    @staticmethod
    def is_valid_day(year, month, day):

        ## 정수가 아니라면 유효하지 않다.
        if not UTIL.isNum(day):
            return False

        ## 윤년일때와 아닐때는 2월의 날이 다르다. 
        ## 각 리스트의 첫번째 인덱스값은 dummy value이다.
        DaysInMonth = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
        DaysInMonthLeap = [0, 31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


        ## 달이 유효하고
        if UTIL.is_valid_month(month):

            ## 윤년이면 
            if UTIL.is_leap(year):

                ## 해당 달에서 나올 수 있는 날이면 유효하다. 
                ## 달을 DaysInMonthLeap 리스트로 체크한다.
                if 1 <= int(day) <= DaysInMonthLeap[int(month)]:
                    return True

            ## 윤년이 아니면
            else:

                ## 해당 달에서 나올 수 있는 날이면 유효하다.
                ## 달을 DaysInMonth 리스트로 체크한다.
                if 1 <= int(day) <= DaysInMonth[int(month)]:
                    return True


        return False

    ## 입력 날짜의 유효성 검사 함수
    @staticmethod
    def is_valid_date(year, month, day):

        ## 년도, 월, 일이 모두 유효하면 유효한 날짜이다.
        if UTIL.is_valid_year(year) and UTIL.is_valid_month(month) and UTIL.is_valid_day(year, month, day):
            
            ## 현재 날짜를 불러온다.
            now = datetime.now()
            
            ## 불러온 현재 날짜를 이용하여 년, 월, 일을 받아온다.
            nowYear = now.year
            nowMonth = now.month
            nowDay = now.day

            
            ## 현재 날짜를 날짜 형식에 맞추어 nowDate에 넣어준다.
            nowDate = time.strptime(f"{nowYear}-{nowMonth}-{nowDay}", "%Y-%m-%d")
            
            ## 입력된 날짜를 날짜 형식에 맞추어 inputDate에 넣어준다.
            inputDate = time.strptime(f"{year}-{month}-{day}", "%Y-%m-%d")
            
            
            ## 입력된 날짜가 현재 날짜보다 더 이전이면 유효한 날짜이다.
            if inputDate < nowDate:
                return 
                ## return "Yes"
            
        
            ## 입력된 날짜가 현재 날짜이거나 그 이후이면 유효하나 정보는 없다.
            else:
                raise ValueError
                ##return "valid but future"

        ## 유효하지 않은 날짜이다.
        raise ValueError
        ##return "not valid"
        
    ## 윤년 체크 함수
    @staticmethod
    def is_leap(year):
        return ( int(year)%400 == 0 ) or ( ( int(year)%4 == 0 ) and ( int(year)%100 != 0 ) )
    
    
    
    
    ## 평균운량을 맑음, 구름조금, 구름 많음, 흐림 중 하나로 표현해주는 함수
    ## 기상청 날씨누리에 평균운량의 해석을 맑음(0~2), 구름조금(3~5), 구름많음(6~8), 흐림(9~10이상)로 한다고 쓰여있다.
    @staticmethod
    def cloud2word(avgCloud):
        
        if 0<= avgCloud < 3:
            return "맑음"
        
        if 3 <= avgCloud < 6:
            return "구름조금"
        
        if 6 <= avgCloud < 9:
            return "구름많음"
        
        if 9 <= avgCloud:
            return "흐림"
        
        
    ## 숫자로 주어진 월을 문자열로 바꿔주는 함수이다.
    @staticmethod
    def month2word(month):
        monthDict = {
                "1" : "Jan",
                "2" : "Feb",
                "3" : "Mar",
                "4" : "Apr",
                "5" : "May",
                "6" : "June",
                "7" : "July",
                "8" : "Aug",
                "9" : "Sep",
                "10": "Oct",
                "11": "Nov",
                "12": "Dec"
        }
        
        return monthDict[month]
    
    
    ## 옷차림을 추천해주기 위한 함수이다.
    @staticmethod
    def weatherClothes(temperature):
            
        if temperature < 5:
            print("-----많이 추움-----")
            print("--패딩")
            print("--두꺼운 코트")
            print("--기모")
            print("--히트택/내복")
            print("--목도리")
            print("--장갑")
            
            
        elif 5 <= temperature < 8:
            print("-----조금 추움-----")
            print("--코트")
            print("--가죽자켓")
            print("--야구점퍼")
            print("--니트+후리스")
            print("--청바지")
            print("--슬랙스")
            
        
        elif 8 <= temperature < 11:
            print("-----조금 추움-----")
            print("--트렌치코트")
            print("--야상")
            print("--자켓")
            print("--청바지")
            print("--슬랙스")
            
            
        elif 11 <= temperature < 16:
            print("-----쌀쌀함-----")
            print("--기모후드티")
            print("--자켓/바람막이")
            print("--가디건")
            print("--니트/맨투맨")
            print("--셔츠")
            print("--청바지")
            print("--슬랙스")
            
        
        elif 16 <= temperature < 19:
            print("-----긴팔입고 활동하기 좋은 날씨-----")
            print("--후드티")
            print("--바람막이")
            print("--가디건")
            print("--니트/맨투맨")
            print("--셔츠")
            print("--청바지")
            print("--슬랙스")
        
        elif 19 <= temperature < 22:
            print("-----반팔입고 활동하기 좋은 날씨-----")
            print("--티셔츠")
            print("--셔츠")
            print("--청바지")
            print("--면바지")
            
        elif 22 <= temperature < 27:
            print("-----약간 더움-----")
            print("--티셔츠")
            print("--린넨셔츠")
            print("--반바지")
            
        elif temperature >= 27:
            print("-----많이 더움-----")
            print("--민소매")
            print("--반팔티")
            print("--반바지")
            
    
    ## 크롬드라이버 초기화과정
    @staticmethod
    def InitChromeDriver(day):
        
        ## 함수 외부에서 드라이버를 이용해서 URL에 접근할 것이기 때문에 전역변수로 둔다.
        global driver
        
        ## 크롬드라이버를 불러온다. 불러오는 중이라는 멘트를 출력해준다.
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        print("크롬드라이버로 URL에 접근 중.. \r", end="")
        driver.implicitly_wait(3)

        # 크롬드라이버로 url에 접근한다. 날씨 정보를 불러오고 있다는 멘트를 출력해주고, 모든 과정이 끝났다면 날씨 정보를 성공적으로 불러왔다는 멘트를 출력한다.
        driver.get("https://www.weather.go.kr/w/index.do")
        print(f"URL 접근 완료! {day} 날씨 정보 불러오는 중.. \r", end="")
        time.sleep(2)
        print(f"{day} 날씨 정보를 성공적으로 불러왔습니다.        ")
        
    
    ## 정보를 크롤링한다.
    @staticmethod
    def crawlInfo():
        
        global html
        global soup
        global temperatures
        global precipitation
        global precipitationPosibility
        
        html = driver.page_source ## html변수에 현재 크롬드라이버가 탐색중인 웹사이트의 소스코드를 저장한다. 
        soup = BeautifulSoup(html, 'html.parser') ## soup변수에 BeautifulSoup을 이용하여 문자열을 이용가능한 객체로 변환시켜준다.
        
        ## temperatures변수에는 soup에 저장된 웹사이트 소스 코드에서 span 태그의 feel 클래스로 쌓여있는 텍스트를 뽑아서 저장한다.
        ## temperatures에는 현재시간 기준 글피/그글피까지의 기온이 저장되어 있다. 현재 시간에 따라 기상청은 단기예보를 글피나 그글피까지 제공한다.
        temperatures = soup.select('span.feel')
        
        ## precipitation변수에는 soup에 저장된 웹사이트 소스 코드에서 ul 태그 안의 li 태그중 5번째 안에 두번째 span 다음 span 태그로 쌓여있는 텍스트를 뽑아서 저장한다.
        ## precipitation에는 현재시간 기준 글피/그글피까지의 강수량정보가 포함되어 있다. 현재 시간에 따라 기상청은 단기예보를 글피나 그글피까지 제공한다.
        precipitation = soup.select('ul > li:nth-child(5) > span+span')
        
        ## precipitationPosibility변수에는 soup에 저장된 웹사이트 소스 코드에서 ul 태그 안의 li 태그중 6번째 안에 두번째 span 다음 span 태그로 쌓여있는 텍스트를 뽑아서 저장한다.
        ## precipitationPosibility에는 현재시간 기준 글피/그글피까지의 강수확률정보가 포함되어 있다. 현재 시간에 따라 기상청은 단기예보를 글피나 그글피까지 제공한다.
        precipitationPosibility = soup.select('ul > li:nth-child(6) > span+span')
        
    
    ## 크롤링된 정보 중 필요한 부분만 골라낸다.
    @staticmethod
    def getInfoFromCrawledInfo(Temps, sensibleTemps, precipitationPosibilities, start, end):
        
        ## 현재 시간부터 글피/그글피까지의 기온정보가 포함되어 있는 temperatures 배열을 돌며 
        for i, temperature in enumerate(temperatures):

            ## 현재 탐색하는 부분이 start번째부터 end번째 사이인지 체크한다. start번째부터 end번째만 불러와야 입력날짜의 날씨 정보만을 골라낼 수 있다.
            if start <= i <= end:
                Temps.append(int(temperature.text[:2])) ## 기온 정보를 Temps 리스트에 append한다.
                sensibleTemps.append(int(temperature.text[4:6])) ## 체감온도 정보를 sensibleTemps 리스트에 append한다.

                ## 강수확률 정보가 없을 때 
                if ord(precipitationPosibility[i].text[0]) == 160:

                    ## 강수량 정보가 없으면 비가 오지 않는다고 가정한다
                    if precipitation[i].text == "-":
                        precipitationPosibilities.append(0)

                    ## 강수량 정보가 있다면
                    else:

                        ## 현재는 오고 있으므로 강수확률을 100%로 설정하였다.
                        if i == 0:
                            precipitationPosibilities.append(100)

                        ## 그 이후 미래에는 강수확률을 90%로 설정하였다.
                        else:
                            precipitationPosibilities.append(90)


                ## 강수확률 정보가 있을 때는 리스트에 추가해준다. 추가하는 원소는 강수확률에서 %를 뺀 한자리 혹은 두자리 정수이다.
                else:
                    precipitationPosibilities.append(int(precipitationPosibility[i].text[:len(precipitationPosibility[i].text)-1]))
                    
                    

In [ ]:
## 정적 크롤링 위해 필요한 모듈 --> 주피터 기본 모듈
import requests
from bs4 import BeautifulSoup

## 동적 크롤링 위해 필요한 모듈 
from selenium import webdriver # --> pip install selenium 통해 직접 설치

# pip install webdriver-manager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import re
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd

from copy import deepcopy
from datetime import datetime, timedelta


## 날씨 정보를 얻기위한 모든 과정을 담고있는 클래스
class Weather:
    
    def __init__(self):
        
        while (True):
            print("\n\n\n<< 날씨 정보를 시각화하기 >>\n")
            print("1. 오늘의 날씨 출력")
            print("2. 내일, 모레, 글피의 날씨 출력")
            print("3. 과거의 날씨 출력")
            print("4. 지정한 날짜의 과거 모든 날씨 출력")
            print("5. 연평균 기온 추이")
            print("6. 구간 최고기온")
            print("7. 구간 최저기온")
            print("8. 내일의 옷차림 추천")
            print("9. 종료")
            
            print()
            
            ## 정수입력이 아닌경우를 거르는 예외처리
            try:
                cmd = int(input("숫자를 입력하시오: "))
                
                
            except:
                print("\n1~9 사이의 정수만 입력해주세요.")
                continue
                
                
            
            # 오늘의 날씨 출력
            if cmd == 1:
                Weather.today_temp(self)
                
            # 내일, 모레, 글피의 날씨 출력
            elif cmd == 2:
                Weather.future_temp(self)
            
            # 과거의 날씨 출력
            elif cmd == 3:
                Weather.past_temp(self)
                
            # 지정한날짜의 과거 모든 날씨 출력
            elif cmd == 4:
                Weather.pastToday_temp(self)
                
            # 연평균 기온 추이
            elif cmd == 5:
                Weather.annual_average_temp(self)
                
            # 구간 최고기온
            elif cmd == 6:
                Weather.max_temp(self)
                
            # 구간 최저기온
            elif cmd == 7:
                Weather.min_temp(self)
            
            # 내일의 옷차림 추천
            elif cmd == 8:
                Weather.recommend_clothes(self)
                
            # 종료
            elif cmd == 9:
                break
                
            ## 1~9 이외의 정수가 입력된 경우를 거르는 조건문
            else:
                print("\n1~9 사이의 정수만 입력해주세요.")
                continue
            
        print("\n\n----------------------------------------------------- 날씨 정보 시각화 프로그램을 종료합니다. -----------------------------------------------------")
        
        return
    
    
    
    
    ## 오늘의 날씨 출력
    ## 함수를 실행하면 현재 시간을 기준을 24시간 이후까지의 기온 및 강수확률 정보를 그래프로 그려준다.
    @checkRunningTime
    def today_temp(self):
        
        ## 크롬드라이버로 날씨 URL에 접근한다.
        UTIL.InitChromeDriver("오늘")
        
        ## 날씨정보를 크롤링해온다.
        UTIL.crawlInfo()
        
        now = datetime.now() ## 현재 시간을 불러온다.
        
        ## 현재시간부부터 24시간 이후까지의 시간정보를 저장하는 리스트이다. 그래프의 x좌표들을 저장하는 과정이다.
        ## 23시 다음에는 24시가 아니라 0시로 넘어가야 하므로 나머지 연산을 이용하였다.
        times = []
        for i in range(24):
            times.append(str((now.hour+1+i)%24))
        times = np.array(times) 
        
        todayTemps = [] ## 현재시간 기준 24시간 이후까지의 기온을 저장하는 리스트
        sensibleTemps = [] ## 현재시간 기준 24시간 이후까지의 체감온도를 저장하는 리스트
        precipitationPosibilities = [] ## 현재시간 기준 24시간 이후까지의 강수확률을 저장하는 리스트
        
        ## 기온, 체감온도, 강수확률 정보를 리스트에 담는다. 각각 24개의 정보가 담길 것이다.
        UTIL.getInfoFromCrawledInfo(todayTemps, sensibleTemps, precipitationPosibilities, 0,23)
        
        
        ## 현재 figure의 사이즈는 20,10이다.
        fig = plt.figure(figsize=(20,10))
        
        ## 2개의 그래프를 그릴 예정이므로 add_subplot을 이용하였다.
        ## tempGraph는 기온과 체감온도 정보를 둘다 그려준다.
        ## preciGraph는 강수확률 정보를 그려준다.
        tempGraph = fig.add_subplot(2,1,1)
        preciGraph = fig.add_subplot(2,1,2)
        
        
        tempGraph.plot(times, todayTemps, label="todayTemp") ## x좌표는 times, y좌표는 todayTemps, 그리고 범례는 todayTemp로 설정해준다.
        tempGraph.plot(times, sensibleTemps, label="sensibleTemp") ## x좌표는 times, y좌표는 sensibleTemps, 그리고 범례는 sensibleTemp로 설정해준다.
        tempGraph.set_title("Today Temperature", size=30) ## 첫번째 그래프의 제목은 Today Temperature로 설정해준다.
        tempGraph.set_xlabel("Hours", size=20) ## x label 값은 시간이다.
        tempGraph.set_ylabel("Temperature", size=20) ## y label 값은 온도이다.
        
        ## 그래프를 보기 좋게 하기 위해 기온, 체감온도의 값들 중 가장 작은 값보다 1 작은수부터, 기온, 체감온도의 값들 중 가장 큰 수보다 1 큰수까지 y값의 범위를 설정하였다.
        tempGraph.set_ylim(min(min(todayTemps), min(sensibleTemps))-1, max(max(todayTemps), max(sensibleTemps))+1)
        tempGraph.legend() ## 범례를 표시한다.
        tempGraph.tick_params(axis='both', which='major', labelsize=15) ## 첫번째 그래프의 x, y label의 폰트 사이즈를 15로 키워준다.
        
        preciGraph.plot(times, precipitationPosibilities) ## x좌표는 times, y좌표는 precipitationPosibilities로 설정해준다.
        preciGraph.set_title("Precipitation Expect", size=30) ## 두번째 그래프의 제목은 Precipitation Expect로 설정해준다.
        preciGraph.set_xlabel("Hours", size=20) ## x label 값은 시간이다.
        preciGraph.set_ylabel("Precipitation rate(%)", size=20) ## y label 값은 강수확률이다.
        preciGraph.set_ylim(min(precipitationPosibilities)-1, max(precipitationPosibilities)+1) ## 강수확률 또한 가장 작은 확률-1 부터 가장 큰확률 +1 까지의 수를 범위로 설정하였다.
        preciGraph.tick_params(axis='both', which='major', labelsize=15) ## 두번째 그래프의 x, y label의 폰트 사이즈를 15로 키워준다.
        
        
        ## tight_layout을 위한 과정이다.
        fig.tight_layout()
        
        ## 그래프가 그려진채로 다음 과정을 진행하기 위해 작성하였다.
        plt.show()
        
        ## pandas를 활용하여 정보를 화면에 보여준다.
        raw_data = {"todayTemp" : todayTemps, "sensibleTemp" : sensibleTemps, "preciExpect" : precipitationPosibilities}
        data = pd.DataFrame(raw_data, index=times)
        print(data)
        
        print("\n\n------------------------------------------------------- 오늘의 날씨 출력 종료 -------------------------------------------------------")
        return
        
    
    ## 내일, 모레, 글피의 날씨 출력
    ## 함수를 실행하면 내일/모레/글피 중 하루를 입력하고, 입력한 날의 오전 1시부터 다음날 0시까지의 기온, 체감온도, 강수확률을 그래프로 보여준다.
    @checkRunningTime
    def future_temp(self):
        
        ## 날씨 정보가 궁금한 날을 입력받는다.
        day = input("내일, 모레, 글피 중 하루를 입력하시오: ")
        
        ## 만약 잘못된 입력이 들어온다면 에러메세지를 출력해주고 함수를 종료한다.
        if day != "내일" and day != "모레" and day != "글피":
            print("잘못된 입력입니다!")
            print(f"\n\n------------------------------------------------------- 미래의 날씨 출력 종료 -------------------------------------------------------")
            return
        
        ### 크롬드라이버로 날씨 URL에 접근한다.
        driver = UTIL.InitChromeDriver(day)
        
        ## 현재 시간을 nowHour에 저장한다.
        now = datetime.now()
        nowHour = now.hour
        
        ## 내일, 모레, 글피의 날씨 정보가 저장되어 있는 리스트의 인덱스를 지정해놓았다.
        ## 크롤링을 통해 받아오는 기온, 체감온도, 강수확률 정보는 현재시간을 기준으로 글피까지 정보가 모두 리스트에 들어 있기 때문에
        ## 내일 -> [24 - 현재시간      : 24 - 현재시간 + 23]
        ## 모레 -> [24 - 현재시간 + 24 : 24 - 현재시간 + 47]
        ## 글피 -> [24 - 현재시간 + 48 : 24 - 현재시간 + 71]
        ## 위 세가지 범위로 슬라이싱 해올 수 있다.
        indexes = {"내일" : (24 - nowHour, 24 - nowHour + 23), "모레": (24 - nowHour + 24, 24 - nowHour + 47), "글피": (24 - nowHour + 48, 24 - nowHour + 71)}
        
        ## 현재 선택한 날짜의 시작시간과 끝 시간을 start, end에 저장해준다.
        start = indexes[day][0]
        end = indexes[day][1]
        
        ## 날씨정보를 크롤링해온다.
        UTIL.crawlInfo()

        
        ## 시간은 오전 1시부터 다음날 0시까지이다. 
        times = [str(i) for i in range(1,24)] + ["0"]
        times = np.array(times)

        Temps = [] ## 입력한 날의 기온을 저장하는 리스트
        sensibleTemps = [] ## 입력한 날의 체감온도를 저장하는 리스트
        precipitationPosibilities = [] ## 입력한 날의 강수확률을 저장하는 리스트
        
        
        ## 기온, 체감온도, 강수확률 정보를 리스트에 담는다. 각각 24개의 정보가 담길 것이다.
        UTIL.getInfoFromCrawledInfo(Temps, sensibleTemps, precipitationPosibilities, start, end)
        
        ## 현재 figure의 사이즈는 20,10이다.
        fig = plt.figure(figsize=(20,10))

        ## 2개의 그래프를 그릴 예정이므로 add_subplot을 이용하였다.
        ## tempGraph는 기온과 체감온도 정보를 둘다 그려준다.
        ## preciGraph는 강수확률 정보를 그려준다.
        tempGraph = fig.add_subplot(2,1,1)
        preciGraph = fig.add_subplot(2,1,2)


        tempGraph.plot(times, Temps, label="Temp") ## x좌표는 times, y좌표는 Temps, 그리고 범례는 Temp로 설정해준다.
        tempGraph.plot(times, sensibleTemps, label="sensibleTemp") ## x좌표는 times, y좌표는 sensibleTemps, 그리고 범례는 sensibleTemp로 설정해준다.
        tempGraph.set_title("Temperature", size=30) ## 첫번째 그래프의 제목은 Temperature로 설정해준다.
        tempGraph.set_xlabel("Hours", size=20) ## x label 값은 시간이다.
        tempGraph.set_ylabel("Temperature", size=20) ## y label 값은 온도이다.
        tempGraph.set_ylim(min(min(Temps), min(sensibleTemps))-1, max(max(Temps), max(sensibleTemps))+1) ## y의 범위는 가장 작은 온도보다 1작은 수 부터 가장 큰 온도보다 1큰수까지이다.
        tempGraph.legend() ## 범례를 표시한다.
        tempGraph.tick_params(axis='both', which='major', labelsize=15) ## 첫번째 그래프의 x, y label의 폰트 사이즈를 15로 키워준다.

        preciGraph.plot(times, precipitationPosibilities) ## x좌표는 times, y좌표는 precipitationPosibilities로 설정해준다.
        preciGraph.set_title("Precipitation Expect", size=30) ## 두번째 그래프의 제목은 Precipitation Expect로 설정해준다.
        preciGraph.set_xlabel("Hours", size=20) ## x label 값은 시간이다.
        preciGraph.set_ylabel("Precipitation rate(%)", size=20) ## y label 값은 강수확률이다.
        preciGraph.set_ylim(min(precipitationPosibilities)-1, max(precipitationPosibilities)+1) ## 강수확률 또한 가장 작은 확률-1 부터 가장 큰확률 +1 까지의 수를 범위로 설정하였다.
        preciGraph.tick_params(axis='both', which='major', labelsize=15) ## 두번째 그래프의 x, y label의 폰트 사이즈를 15로 키워준다.


        ## tight_layout을 위한 과정이다.
        fig.tight_layout()

        ## 그래프가 그려진채로 다음 과정을 진행하기 위해 작성하였다.
        plt.show()
        
        print(f"\n\n------------------------------------------------------- {day}의 날씨 출력 종료 -------------------------------------------------------")

        return
    
    
    ## 과거의 날씨 출력
    ## 함수를 실행하면 과거의 년원일을 입력받아, 그 날의 평균기온, 최고기온, 최저기온, 평균운량, 강수량을 모두 화면에 출력한다.
    @checkRunningTime
    def past_temp(self):
        
        ## 년월일을 입력받는다.
        print("1960년 1월 1일 ~ 어제날짜까지 입력가능\n")
        year = input("년도를 입력하시오: ")
        month = input("월을 입력하시오: ")
        day = input("일을 입력하시오: ")
        
        try:
            UTIL.is_valid_date(year, month, day)
            
        except:
            ## 에러메세지와 함께 종료메세지를 출력해준다.
            print("\n\n날짜 입력 오류. 날짜 범위(1960년 1월 1일 ~ 어제)만 입력이 가능합니다.")
            print(f"\n\n------------------------------------------------------- 과거 날씨 출력 종료 -------------------------------------------------------")
            return
            
            
        
        ## 날짜를 정수형으로 변환해주어 계산이 편리하도록 해준다.
        day = int(day)

        
        ## 크롤링을 할 URL을 문자열로 저장한다.
        URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+year+"&mm="+month+"&obs=1"

        ## HTTP GET Request
        req = requests.get(URL)
        
        ## HTML 소스 코드를 문자열 형식으로 가져온다
        html = req.text
        
        ## BeautifulSoup으로 문자열 형식의 html소스를 이용가능한 python객체로 변환한다.
        ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시한다.
        ## 이번 날씨 시각화 프로젝트에서는 Python 내장 html.parser를 이용한다.
        soup = BeautifulSoup(html, 'html.parser')
        
        
        ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
        weathers = soup.select("td > span")

        
        ## avgTemp : 평균기온을 뽑아내는 정규표현식
        ## maxTemp : 최고기온을 뽑아내는 정규표현식
        ## minTemp : 최저기온을 뽑아내는 정규표현식
        ## avgCloud : 평균운량을 뽑아내는 정규표현식
        ## precipitation : 일강수량을 뽑아내는 정규표현식
        avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")
        maxTemp = re.compile(r"(?<=최고기온:).*(?=℃)")
        minTemp = re.compile(r"(?<=최저기온:).*(?=℃)")
        avgCloud = re.compile(r"(?<=평균운량:).*")
        precipitation = re.compile(r"(?<=일강수량:).*")

        
        
        ## avgTemps : 해당 달의 평균기온을 모두 저장하는 리스트
        ## maxTemps : 해당 달의 최고기온을 모두 저장하는 리스트
        ## minTemps : 해당 달의 최저기온을 모두 저장하는 리스트
        ## avgClouds : 해당 달의 평균운량을 모두 저장하는 리스트
        ## precipitations : 해당 달의 일강수량을 모두 저장하는 리스트
        avgTemps = [0]
        maxTemps = [0]
        minTemps = [0]
        avgClouds = [0]
        precipitations = [0]

        ## 며칠 째 탐색중인지 체크하기 위한 dayCount 변수이다.
        dayCount = 0
        
        ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
        for weather in weathers:
            
            ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
            ## maxTempSearch : maxTemp 정규식으로 평균기온을 찾는다.
            ## minTempSearch : minTemp 정규식으로 평균기온을 찾는다.
            ## avgCloudSearch : avgCloud 정규식으로 평균기온을 찾는다.
            ## precipitationSearch : precipitation 정규식으로 평균기온을 찾는다.
            avgTempSearch = avgTemp.search(weather.text)
            maxTempSearch = maxTemp.search(weather.text)
            minTempSearch = minTemp.search(weather.text)
            avgCloudSearch = avgCloud.search(weather.text)
            precipitationSearch = precipitation.search(weather.text)                               
            
            
            ## 평균기온이 찾아졌다면, avgTemps 리스트에 평균기온을 추가해준다.
            if avgTempSearch:
                avgTemps.append(float(avgTempSearch.group()))

            ## 최고기온이 찾아졌다면, maxTemps 리스트에 평균기온을 추가해준다.
            elif maxTempSearch:
                maxTemps.append(float(maxTempSearch.group()))

            ## 최저기온이 찾아졌다면, minTemps 리스트에 평균기온을 추가해준다.
            elif minTempSearch:
                minTemps.append(float(minTempSearch.group()))
                
            ## 평균운량이 찾아졌다면, avgClouds 리스트에 평균기온을 추가해준다.
            elif avgCloudSearch:
                avgClouds.append(float(avgCloudSearch.group()))
                
            ## 일강수량이 찾아졌다면, precipitations 리스트에 평균기온을 추가해준다.
            elif precipitationSearch:
                
                ## "-"는 정보가 없다는 것을 의미한다.
                if precipitationSearch.group() == " - ":
                    precipitations.append("강수 정보 없음")
                    
                ## 강수정보가 0.0mm이면 비가 오지 않았다는 것을 의미한다.
                elif precipitationSearch.group() == "0.0mm":
                    precipitations.append("비 오지 않음")
                    
                
                ## 나머지는 모두 비가 온 상황이므로 강수량을 precipitations 리스트에 append해준다.
                else:
                    precipitations.append(precipitationSearch.group())
                
                
                ## 제공하는 정보는 총 5개이므로, 이들이 모두 탐색되었다면 다음 날짜로 넘어간다.
                dayCount += 1


            ## 만약, 정보를 받은 날짜의 개수와 입력날짜가 같다면 날씨 정보를 출력해주고 반복문을 빠져나온다.
            if dayCount == day:
                print(f"\n\n<<{year}년 {month}월 {day}일의 날씨>>")

                print(f"평균기온 : {avgTemps[dayCount]}℃")
                print(f"최고기온 : {maxTemps[dayCount]}℃")
                print(f"최저기온 : {minTemps[dayCount]}℃")
                print(f"평균운량 : {avgClouds[dayCount]}({UTIL.cloud2word(avgClouds[dayCount])})")
                print(f"일강수량 : {precipitations[dayCount]}")

                break
                
            
        ## 매서드 종료 메세지를 출력해준다.
        print(f"\n\n------------------------------------------------------- 과거 날씨 출력 종료 -------------------------------------------------------")
        return
    
    ## 지정한 날짜의 과거 모든 날씨 출력
    ## 함수를 실행하면 월일을 입력한다. 그렇게 입력된 월일에 대해 1961년부터 2022년까지의모든 날씨정보를 불러와서 평균기온, 최고기온, 최저기온, 평균운량을 모두 그래프로 나타내준다.
    @checkRunningTime
    def pastToday_temp(self):
        month = input("__월을 입력하시오: ")
        day = input("__일을 입력하시오: ")
        print()
        
    
        ## avgTemps : 1960년부터 현재까지 년도별 평균기온 저장 리스트
        ## maxTemps : 1960년부터 현재까지 년도별 최고기온 저장 리스트
        ## minTemps : 1960년부터 현재까지 년도별 최저기온 저장 리스트
        ## avgClouds : 1960년부터 현재까지 년도별 평균운량 저장 리스트
        ## days : 1960년부터 현재까지 년도 저장 리스트 (2월 29일이 입력된 경우, 모든 년도가 이 리스트에 들어가지는 않는다.)
        avgTemps = []
        maxTemps = []
        minTemps = []
        avgClouds = []
        days = []
        
        ## 1960년부터 2022년까지 체크한다.
        for year in range(1960, 2023):
            year = str(year)
            month = str(month)
            day = str(day)
            
            ## 년월일 유호성 검사를 진행한다.
            try:
                UTIL.is_valid_date(year, month, day)
                
            except:
                
                ## 2월 29일의 경우에는 윤년에 대해서는 계산이 가능하다.
                if (month == "2" and day == "29"):
                    continue
                
                ## 올해 아직 오지 않은 날짜이므로 유효하진 않지만 프로그램은 이 전까지의 날짜들 정보를 표시해주면 된다.
                if year == "2022":
                    break
                
                ## 에러메세지와 함께 종료메세지를 출력해준다.
                print("\n\n날짜 오류. 날짜 범위(1월 1일 ~ 12월 31일)만 입력이 가능합니다.")
                print(f"\n\n------------------------------------------------------- 지정한 날짜의 과거 모든 날씨 출력 종료 -------------------------------------------------------")
                
                return
        
            
            ## 계산의 편의성을 위해 정수형으로 변환해준다.
            day = int(day)
                
            
            ## 크롤링을 할 URL을 문자열로 저장한다.
            URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+year+"&mm="+month+"&obs=1"

            ## HTTP GET Request
            req = requests.get(URL)

            ## HTML 소스 문자열 형식으로 가져온다
            html = req.text

            ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환한다
            ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시한다.
            ## 이번 날씨 정보 시각화 프로젝트에서는 Python 내장 html.parser를 이용한다.
            soup = BeautifulSoup(html, 'html.parser')


            ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
            weathers = soup.select("td > span")
            
            
            ## avgTemp : 평균기온을 뽑아내는 정규표현식
            ## maxTemp : 최고기온을 뽑아내는 정규표현식
            ## minTemp : 최저기온을 뽑아내는 정규표현식
            ## avgCloud : 평균운량을 뽑아내는 정규표현식
            avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")
            maxTemp = re.compile(r"(?<=최고기온:).*(?=℃)")
            minTemp = re.compile(r"(?<=최저기온:).*(?=℃)")
            avgCloud = re.compile(r"(?<=평균운량:).*")
            
            
            ## 며칠 째 탐색중인지 체크하기 위한 dayCount 변수이다.
            dayCount = 0

            ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
            for weather in weathers:

                ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                ## maxTempSearch : maxTemp 정규식으로 평균기온을 찾는다.
                ## minTempSearch : minTemp 정규식으로 평균기온을 찾는다.
                ## avgCloudSearch : avgCloud 정규식으로 평균기온을 찾는다.
                avgTempSearch = avgTemp.search(weather.text)
                maxTempSearch = maxTemp.search(weather.text)
                minTempSearch = minTemp.search(weather.text)
                avgCloudSearch = avgCloud.search(weather.text)                             


                ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                if avgTempSearch:
                    nowAvgTemp = float(avgTempSearch.group())

                ## 최고기온이 찾아졌다면, nowMaxTemp에 최고기온을 저장해준다.
                elif maxTempSearch:
                    nowMaxTemp = float(maxTempSearch.group())

                ## 최저기온이 찾아졌다면, nowMinTemp에 최저기온을 저장해준다.
                elif minTempSearch:
                    nowMinTemp = float(minTempSearch.group())

                ## 평균운량이 찾아졌다면, nowAvgCloud에 평균운량을 저장해준다.
                elif avgCloudSearch:
                    nowAvgCloud = float(avgCloudSearch.group())
                    dayCount += 1



                ## 만약, 정보를 받은 날짜의 개수와 입력날짜가 같다면 날씨 정보를 출력해주고 반복문을 빠져나온다.
                if dayCount == day:
                    avgTemps.append(nowAvgTemp)
                    maxTemps.append(nowMaxTemp)
                    minTemps.append(nowMinTemp)
                    avgClouds.append(nowAvgCloud)
                    
                    days.append(year)
                        
                    print(f"{year}년 {month}월 {day}일 정보 불러오기 완료 \r", end="")
                    
                    break

        
        print("모든 정보를 성공적으로 불러왔습니다.         ")
                    
        ## 4개의 그래프의 위치를 정해준다.
        fig = plt.figure(figsize=(10,20));
        avgTempsGraph = fig.add_subplot(4,1,1)
        maxTempsGraph = fig.add_subplot(4,1,2)
        minTempsGraph = fig.add_subplot(4,1,3)
        avgCloudsGraph = fig.add_subplot(4,1,4)

        ## 평균기온 그래프
        avgTempsGraph.plot(days, avgTemps, "go-")
        avgTempsGraph.set_title(f"Average Temperature of {UTIL.month2word(month)} {day}", size=30)
        avgTempsGraph.set_xlabel("Years", size=20)
        avgTempsGraph.set_ylabel("Average Temperatures", size=20)
        avgTempsGraph.set_ylim([min(avgTemps)-1, max(avgTemps)+1])
        avgTempsGraph.tick_params(axis = 'x', size=10,labelrotation =90)
        
        
        
        ## 최고기온 그래프
        maxTempsGraph.plot(days, maxTemps, "ro-")
        maxTempsGraph.set_title(f"Max Temperature of {UTIL.month2word(month)} {day}", size=30)
        maxTempsGraph.set_xlabel("Years", size=20)
        maxTempsGraph.set_ylabel("Max Temperatures", size=20)
        maxTempsGraph.set_ylim([min(maxTemps)-1, max(maxTemps)+1])
        maxTempsGraph.tick_params(axis = 'x', labelrotation =90)
        
        
        ## 최저기온 그래프
        minTempsGraph.plot(days, minTemps, "bo-")
        minTempsGraph.set_title(f"Min Temperature of {UTIL.month2word(month)} {day}", size=30)
        minTempsGraph.set_xlabel("Years", size=20)
        minTempsGraph.set_ylabel("Min Temperatures", size=20)
        minTempsGraph.set_ylim([min(minTemps)-1, max(minTemps)+1])
        minTempsGraph.tick_params(axis = 'x', labelrotation =90)
        
        
        ## 평균운량 그래프
        avgCloudsGraph.plot(days, avgClouds, "o-", color="dodgerblue")
        avgCloudsGraph.set_title(f"Average Cloud of {UTIL.month2word(month)} {day}", size=30)
        avgCloudsGraph.set_xlabel("Years", size=20)
        avgCloudsGraph.set_ylabel("Average Cloud", size=20)
        avgCloudsGraph.set_ylim([min(avgClouds)-1, max(avgClouds)+1])
        avgCloudsGraph.tick_params(axis = 'x', labelrotation =90)
        
        ## tight_layout을 위해서 작성하였다.
        fig.tight_layout()
        
        ## 그래프를 화면에 계속 유지시킨다.
        plt.show()
        
        ## 매서드 종료 메세지를 출력해준다.
        print(f"\n\n------------------------------------------------------- 지정한 날짜의 과거 모든 날씨 출력 종료 -------------------------------------------------------")
        return
        
        
        
    ## 연평균 추이 그래프 출력
    ## 함수를 실행하면 기상청 날씨누리에서 모든 날짜의 평균기온을 받아와서, 년도별로 평균을 낸 후, 각 년도별 평균기온을 그래프로 나타낸다.
    @checkRunningTime
    def annual_average_temp(self):
        
        
        print("연평균 기온 체크 시작! \r", end="")
        Years = np.arange(1960, 2022)
        
        ## avgTemps : 년도별 평균기온이 저장되는 리스트
        ## curYearTemps : 현재 탐색중인 년도의 모든 달의 평균기온이 저장되는 리스트. 저장된 값을 평균내에서 avgTemps에 하나씩 추가한다.
        avgTemps = []
        curYearTemps = []
        
        ## 현재 탐색하는 달의 일 수
        dayCount = 1
        
        
        ## 탐색 시작일 : 1960년 1월 1일
        ## 탐색 종료일 : 2021년 12월 31일 (반복문을 성공적으로 돌기 위해서는 Day2를 2022년 1월 1일로 설정해주어야 한다.)
        Day1 = datetime(1960, 1, 1)
        Day2 = datetime(2022, 1, 1)
        
        
        # Day1과 Day2 사이의 모든 날씨들을 받아온다.
        while (True):
            
            nowYear = Day1.year
            nowMonth = Day1.month
            nowDay = Day1.day
            
            ## 크롤링을 할 URL을 문자열로 저장한다.
            URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+str(nowYear)+"&mm="+str(nowMonth)+"&obs=1"

            ## HTTP GET Request
            req = requests.get(URL)

            ## HTML 소스 문자열 형식으로 가져오기
            html = req.text

            ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환한다.
            ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시한다.
            ## 이번 프로젝트에서는 Python 내장 html.parser를 이용한다.
            soup = BeautifulSoup(html, 'html.parser')


            ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
            weathers = soup.select("td > span")
            
            
            ## avgTemp : 평균기온을 뽑아내는 정규표현식
            avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")
            
            
            ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
            for weather in weathers:
                
                nowYear = Day1.year
                nowMonth = Day1.month
                nowDay = Day1.day

                ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                avgTempSearch = avgTemp.search(weather.text)
                
                ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                if avgTempSearch:

                    curYearTemps.append(float(avgTempSearch.group()))
                    dayCount += 1
                    
                    
                    ## 모든 날짜를 탐색한 경우 그래프를 그리고 함수를 종료한다.
                    if (Day1 == Day2):
                        
                        print("모든 정보를 성공적으로 불러왔습니다.                  ")
                        
                        plt.plot(Years, avgTemps)
                        plt.title(f"Annual Average Temperature")
                        plt.xlabel("years", size=20)
                        plt.ylabel("Average Temperature", size=20)
                        plt.ylim(min(avgTemps)-1, max(avgTemps)+1)
                        plt.xticks(rotation=90)
                        
                        ## 화면에서 계속 유지한다.
                        plt.show()
                        
                        ## 매서드 종료 메세지를 출력해준다.
                        print(f"\n\n------------------------------------------------------- 연평균 기온 추이 정보 출력 종료 -------------------------------------------------------")
                        return
                    
                    nowDay += 1
                    Day1 += timedelta(days=1)
            
            ## 1960년 9월의 경우 30일이 아닌 17일까지밖에 정보가 제공되지 않는다. 이러한 상황에서도 다음달로 탐색을 넘기기 위한 장치이다.
            while Day1.day != 1:
                Day1 += timedelta(days=1)
            
            Day1 -= timedelta(days=1)
            print(f"연평균 기온 체크중.. {Day1.year}년 {Day1.month}월의 정보 불러오기 완료 \r", end="")
            Day1 += timedelta(days=1)
            
            ## 다음년도로 넘어왔으면, 현재 년도의 모든 달의 평균기온이 저장된 curYearTemps를 평균내어, avgTemps에 추가해준다.
            if (Day1.month == 1):
                curYearTemps = np.array(curYearTemps)
                curYearAvgTemp = np.sum(curYearTemps) / dayCount
                avgTemps.append(curYearAvgTemp)
                dayCount = 1
                curYearTemps = []
                
        
        
    
    
    
    ## 구간 최고기온 출력
    ## 함수를 실행하면, 두개의 날짜를 입력받고, 두 날짜 모두 유효하다면 두 날짜 사이의 모든 날들의 평균기온을 불러와서, 그래프로 나타내고 최고기온과 최고기온을 기록한 날짜를 알려준다.
    @checkRunningTime
    def max_temp(self):
        
        ## avgTemps : 각 날짜별 평균기온이 저장되는 리스트
        ## days : 날짜들이 저장되는 리스트 
        avgTemps = []
        days = []
        
        print("두 개의 날짜를 입력하세요.")
        print("ex. 첫번째 날: 2022년 5월 15일, 두번째 날: 2022년 6월 3일\n")
        
        year1 = input("__년을 입력하시오: ")
        month1 = input("__월을 입력하시오: ")
        day1 = input("__일을 입력하시오: ")
        print()
        
        year2 = input("__년을 입력하시오: ")
        month2 = input("__월을 입력하시오: ")
        day2 = input("__일을 입력하시오: ")
        print()
        
        
        ## 두 날짜의 유효성 검사를 진행한다.
        try:
            UTIL.is_valid_date(year1, month1, day1)
            UTIL.is_valid_date(year2, month2, day2)
            
        except:
            print("날짜 오류. 두 날짜 모두 날짜 범위(1960년 1월 1일 ~ 어제)만 입력이 가능합니다.")
            ## 매서드 종료 메세지를 출력해준다.
            print(f"\n\n------------------------------------------------------- 최고기온 기록한 날짜 및 날씨 정보 종료 -------------------------------------------------------")
            return 
            
        
        ## 입력된 날짜들을 날짜 객체로 변환해준다.
        Day1 = datetime(int(year1), int(month1), int(day1))
        Day2 = datetime(int(year2), int(month2), int(day2))
        
        ## 먼저 입력한 날짜가 더 나중 날짜라면 둘을 바꿔준다.
        if Day1 > Day2:
            Day1, Day2 = Day2, Day1
            year1, year2 = year2, year1
            month1, month2 = month2, month1
            day1, day2 = day2, day1
        
        # Day1과 Day2 사이의 모든 날씨들을 받아온다.
        while (True):
            
            nowYear = Day1.year
            nowMonth = Day1.month
            nowDay = Day1.day
            
            ## 크롤링을 할 URL을 문자열로 저장한다.
            URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+str(nowYear)+"&mm="+str(nowMonth)+"&obs=1"

            ## HTTP GET Request
            req = requests.get(URL)

            ## HTML 소스 문자열 형식으로 가져오기
            html = req.text

            ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환하기
            ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시.
            ## 이번 프로젝트에서는 Python 내장 html.parser를 이용했다.
            soup = BeautifulSoup(html, 'html.parser')


            ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
            weathers = soup.select("td > span")
            
            
            ## avgTemp : 평균기온을 뽑아내는 정규표현식
            avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")
            
            
            ## 며칠째인지 체크하기
            dayCount = 1
            
            ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
            for weather in weathers:
                
                nowYear = Day1.year
                nowMonth = Day1.month
                nowDay = Day1.day

                ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                avgTempSearch = avgTemp.search(weather.text)
                
                ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                if avgTempSearch:
                    
                    ## 시작날짜의 달이면서 아직 체크해야 하는 일까지 오지 않았을 때는 dayCount만 1 증가시켜준다.
                    if (nowMonth == month1 and dayCount < nowDay):
                        dayCount += 1
                        continue
                    
                    
                    ## 평균기온을 리스트에 넣어주고, 성공했다면 성공메세지를 출력해준다.
                    avgTemps.append(float(avgTempSearch.group()))
                    days.append(f"{nowYear}/{nowMonth}/{nowDay}")
                    print(f"{Day1.year}년 {Day1.month}월 {Day1.day}일 정보 불러오기 완료 \r", end="")
                    
                    
                    ## 날짜 탐색이 마무리되었다면, 주어진 구간의 평균기온이 나타난 그래프를 그려주고, 최저기온과 최저기온을 기록한 날짜를 출력해준다.
                    if (Day1 == Day2):
                        
                        print("모든 정보를 성공적으로 불러왔습니다.             ")
                        
                        ## 평균기온 그래프
                        plt.plot(days, avgTemps, "g-")
                        plt.title(f"Average Temperature {year1}/{month1}/{day1}~{year2}/{month2}/{day2}")
                        plt.xlabel("years", size=20)
                        plt.ylabel("Average Temperature", size=20)
                        plt.ylim(min(avgTemps)-1, max(avgTemps)+1)
                        plt.xticks(rotation=90)
                        
                        ## 최고기온을 얻고, 그것을 기록한 날짜를 얻어낸다.
                        maxTemp = max(avgTemps)
                        index = avgTemps.index(maxTemp)
                        maxTempDay = days[index]
                        
                        print(f"\n\n{year1}/{month1}/{day1} ~ {year2}/{month2}/{day2} 에서...\n")
                        print(f"가장 더웠던 날: {maxTempDay}")
                        print(f"최고기온 : {maxTemp}")
                        
                        ## 화면에서 계속 유지한다.
                        plt.show()
                        
                        ## 매서드 종료 메세지를 출력해준다.
                        print(f"\n\n------------------------------------------------------- 최고기온 기록한 날짜 및 날씨 정보 종료 -------------------------------------------------------")
                        return
                    
                    nowDay += 1
                    Day1 += timedelta(days=1)
                    
        
        
        
        
    
    ## 구간 최저기온 출력
    ## 함수를 실행하면, 두개의 날짜를 입력받고, 두 날짜 모두 유효하다면 두 날짜 사이의 모든 날들의 평균기온을 불러와서, 그래프로 나타내고 최저기온과 최저기온을 기록한 날짜를 알려준다.
    @checkRunningTime
    def min_temp(self):
        
        ## avgTemps : 각 날짜별 평균기온이 저장되는 리스트
        ## days : 날짜들이 저장되는 리스트 
        avgTemps = []
        days = []
        
        print("두 개의 날짜를 입력하세요.")
        print("ex. 첫번째 날: 2022년 5월 15일, 두번째 날: 2022년 6월 3일\n")
        
        year1 = input("__년을 입력하시오: ")
        month1 = input("__월을 입력하시오: ")
        day1 = input("__일을 입력하시오: ")
        print()
        
        year2 = input("__년을 입력하시오: ")
        month2 = input("__월을 입력하시오: ")
        day2 = input("__일을 입력하시오: ")
        print()
        
        ## 두 날짜의 유효성 검사를 진행한다.
        try:
            UTIL.is_valid_date(year1, month1, day1)
            UTIL.is_valid_date(year2, month2, day2)
            
        except:
            print("날짜 오류. 두 날짜 모두 날짜 범위(1960년 1월 1일 ~ 어제)만 입력이 가능합니다.")
            
            ## 매서드 종료 메세지를 출력해준다.
            print(f"\n\n------------------------------------------------------- 최저기온 기록한 날짜 및 날씨 정보 종료 -------------------------------------------------------")
            return 
        
            
        ## 입력된 날짜들을 날짜 객체로 변환해준다.
        Day1 = datetime(int(year1), int(month1), int(day1))
        Day2 = datetime(int(year2), int(month2), int(day2))
        
        ## 먼저 입력한 날짜가 더 나중 날짜라면 둘을 바꿔준다.
        if Day1 > Day2:
            Day1, Day2 = Day2, Day1
            year1, year2 = year2, year1
            month1, month2 = month2, month1
            day1, day2 = day2, day1
        
        # Day1과 Day2 사이의 모든 날씨들을 받아온다.
        while (True):
            
            nowYear = Day1.year
            nowMonth = Day1.month
            nowDay = Day1.day
            
            ## 크롤링을 할 URL을 문자열로 저장한다.
            URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+str(nowYear)+"&mm="+str(nowMonth)+"&obs=1"

            ## HTTP GET Request
            req = requests.get(URL)

            ## HTML 소스 문자열 형식으로 가져오기
            html = req.text

            ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환하기
            ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시.
            ## 이번 프로젝트에서는 Python 내장 html.parser를 이용했다.
            soup = BeautifulSoup(html, 'html.parser')


            ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
            weathers = soup.select("td > span")
            
            
            ## avgTemp : 평균기온을 뽑아내는 정규표현식
            avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")
            
            
            ## 며칠째인지 체크하기
            dayCount = 1
            
            ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
            for weather in weathers:
                
                nowYear = Day1.year
                nowMonth = Day1.month
                nowDay = Day1.day

                ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                avgTempSearch = avgTemp.search(weather.text)
                
                ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                if avgTempSearch:
                    
                    ## 시작날짜의 달이면서 아직 체크해야 하는 일까지 오지 않았을 때는 dayCount만 1 증가시켜준다.
                    if (nowMonth == month1 and dayCount < nowDay):
                        dayCount += 1
                        continue
                    
                    ## 평균기온을 리스트에 넣어주고, 성공했다면 성공메세지를 출력해준다.
                    avgTemps.append(float(avgTempSearch.group()))
                    days.append(f"{nowYear}/{nowMonth}/{nowDay}")
                    print(f"{Day1.year}년 {Day1.month}월 {Day1.day}일 정보 불러오기 완료 \r", end="")
                    
                    
                    ## 날짜 탐색이 마무리되었다면, 주어진 구간의 평균기온이 나타난 그래프를 그려주고, 최저기온과 최저기온을 기록한 날짜를 출력해준다.
                    if (Day1 == Day2):
                        
                        print("모든 정보를 성공적으로 불러왔습니다.             ")
                        
                        ## 평균기온 그래프
                        plt.plot(days, avgTemps, "g-")
                        plt.title(f"Average Temperature {year1}/{month1}/{day1}~{year2}/{month2}/{day2}")
                        plt.xlabel("years", size=20)
                        plt.ylabel("Average Temperature", size=20)
                        plt.ylim(min(avgTemps)-1, max(avgTemps)+1)
                        plt.xticks(rotation=90)
                        
                        ## 최저기온을 얻고, 그것을 기록한 날짜를 얻어낸다.
                        minTemp = min(avgTemps)
                        index = avgTemps.index(minTemp)
                        minTempDay = days[index]
                        
                        print(f"\n\n{year1}/{month1}/{day1} ~ {year2}/{month2}/{day2} 에서...\n")
                        print(f"가장 추웠던 날: {minTempDay}")
                        print(f"최저기온 : {minTemp}")
                        
                        ## 화면에서 계속 유지한다.
                        plt.show()
                        
                        ## 매서드 종료 메세지를 출력해준다.
                        print(f"\n\n------------------------------------------------------- 최저기온 기록한 날짜 및 날씨 정보 종료 -------------------------------------------------------")
                        return
                    
                    nowDay += 1
                    Day1 += timedelta(days=1)
    
    
    ## 내일의 옷차림 추천 매서드
    ## 함수를 실행하면 1961년부터 올해(2022)에 대해 오늘날짜를 기준으로 HOW_PAST일 전 까지의 모든 평균기온을 모두 받아온 후 리스트에 저장한다.
    ## 이렇게 되면 81개의 리스트가 완성되게 된다. 각 리스트의 크기는 HOW_PAST이다.
    ## 그 이후, 올해 평균기온 리스트과 과거의 평균기온 리스트들의 유사도를 분석하여 가장 유사한 3개의 년도를 얻어낸 후, 과거 선택한 년도의 내일날짜의 평균기온을 받아온다.
    ## 이렇게 얻어온 3개의 평균기온에 대해 유사도를 반영하여 가중평균을 내어서 내일의 기온을 예측한다.
    ## 날씨는 카오스 이론을 따르므로 미세한 차이로도 예측이 크게 빗나갈 수 있지만, 지수적인 차이를 보이기 때문에, 자료가 유사할수록 내일날씨 예측이 더 정확할 수 있다는 것에 정당성이 생긴다.
    @checkRunningTime
    def recommend_clothes(self):
        
        ## 
        temps = []
        NOW_TEMPS = []
        PAST_TEMPS_DICT = {}
        
        ## 며칠 전까지 분석할 지 결정해주는 부분이다.
        HOW_PAST = 180
        
        ## 현재 날짜를 얻어온다.
        now = datetime.now()
        
        ## 올해부터 1961년까지의 모든 년도에 대해 탐색한다.
        for year in range(now.year, 1960, -1):
            
            
            NOW = datetime(year, now.month, now.day) ## 현재 날짜를 날짜객체로 변환
            fr = NOW - timedelta(days=HOW_PAST - 1) ## 현재 날짜 기준 HOW_PAST - 1 일 전의 날짜를 날짜객체로 변환
                
            # fr과 NOW 사이의 모든 날씨들을 받아온다.
            while (fr != NOW):

                nowYear = fr.year
                nowMonth = fr.month
                nowDay = fr.day

                ## 크롤링을 할 URL을 문자열로 저장한다.
                URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+str(nowYear)+"&mm="+str(nowMonth)+"&obs=1"

                ## HTTP GET Request
                req = requests.get(URL)

                ## HTML 소스 문자열 형식으로 가져오기
                html = req.text

                ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환한다.
                ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시한다.
                ## 이번 프로젝트에서는 Python 내장 html.parser를 이용한다.
                soup = BeautifulSoup(html, 'html.parser')


                ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
                weathers = soup.select("td > span")


                ## avgTemp : 평균기온을 뽑아내는 정규표현식
                avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")


                ## 며칠째인지 체크하기
                dayCount = 1

                ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
                for weather in weathers:

                    nowYear = fr.year
                    nowMonth = fr.month
                    nowDay = fr.day

                    ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                    avgTempSearch = avgTemp.search(weather.text)

                    ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                    if avgTempSearch:

                        ## 시작날짜의 달이면서 아직 체크해야 하는 일까지 오지 않았을 때는 dayCount만 1 증가시켜준다.
                        if (nowMonth == NOW.month and dayCount < nowDay):
                            dayCount += 1
                            continue

                        ## 현재 탐색중인 날짜의 평균기온을 temps배열에 추가해주고, 성공 메세지를 출력해준다.
                        temps.append(float(avgTempSearch.group()))
                        print(f"{fr.year}년 {fr.month}월 {fr.day}일 정보 불러오기 완료 \r" , end="")
                        sys.stdout.flush()
                        
                        ## 탐색이 완료되었다면 루프를 탈출한다.
                        if (fr == NOW):
                            break

                        fr += timedelta(days=1)

            print(f"{fr.year}년 {fr.month}월 {fr.day}일 정보 불러오기 완료 \r" , end="")
            sys.stdout.flush()
            
            ## 올해에 대해서는 temps 배열을 NOW_TEMPS로 복사해준다.
            if year == now.year:
                temps.append(temps[-1])
                NOW_TEMPS = deepcopy(temps)
                NOW_TEMPS = np.array(NOW_TEMPS)
            
            ## 과거 년도에 대해서는 temps배열을 PAST_TEMPS_DICTS에 year을 키로 주고 넣어준다.
            else:
                if len(temps) != HOW_PAST:
                    temps.append(temps[-1])
                
                PAST_TEMPS_DICT[year] = np.array(temps)
                
            print(f"{fr.year}년 정보 불러오기 완료             ")
            temps = []
                
        
        ################################################### 현재 정보와 이전 정보를 모두 불러오기 #####################
        
        
        print("\n\n<<내일 날씨 예측 방법을 선택해주세요.>>")
        
        ## 날씨 예측할 세가지 유사도 방법 중 하나를 선택한다.
        while True:
            print("1. Euclidean Similarity")
            print("2. Cosine Similarity")
            print("3. My Similarity")
            print("4. 종료")
            print()
            
            try:
                command = int(input("방법을 선택하시오: "))
                
            except:
                print("1~4 사이의 정수를 입력하세요.\n\n")
                continue
            


            ## 1. Euclidean Similarity
            ## --> 올해 평균기온들과 과거 평균기온들의 차이를 모두 합한 것을 이용하여, 이 값이 작으면 작을수록 유사도가 크다고 생각한다.
            if command == 1:
                
                ## 올해 평균기온 정보와 과거 평균기온 정보의 차이를 저장하는 딕셔너리이다.
                DIFF = {}
                
                ## 과거의 정보들을 모두 돌며
                for year, temps in PAST_TEMPS_DICT.items():
                    
                    ## 올해의 평균기온 정보와 과거의 평균기온 정보의 차이의 합을 저장한 후
                    ## DIFF 딕셔너리에 저장해준다.
                    diff = abs(NOW_TEMPS - temps).sum()
                    DIFF[year] = diff
                
                ## 딕셔너리를 튜플 리스트형태로 바꾼 후, 차이를 기준으로 오름차순 정렬해준다.
                DIFF = sorted(DIFF.items(), key=lambda x : x[1])

                ## 정렬된 DIFF 배열의 가장 앞 세개의 자료가 올해 년도와 가장 유사한 자료들이다.
                top1 = DIFF[0]
                top2 = DIFF[1]
                top3 = DIFF[2]
                

            ## 2. Cosine Similarity
            ## --> 올해 평균기온들과 과거 평균기온들을 벡터로 보고, 두 벡터 사이의 각 차이를 이용하여, 이 값이 작을수록 유사도가 크다고 생각한다.
            elif command == 2:
                
                ## 올해 평균기온 정보 벡터와 과거 평균기온 정보 벡토 사이의 각 차이를 저장하는 딕셔너리이다.
                COSINE = {}
                
                ## 올해 평균기온 정보 벡터의 크기이다.
                magnitudeNow = np.linalg.norm(NOW_TEMPS)
                
                ## 과거의 정보들을 모두 돌며
                for year, temps in PAST_TEMPS_DICT.items():
                    
                    magnitudePast = np.linalg.norm(temps) ## 과거 평균기온 정보 벡터의 크기이다.
                    dot_product = temps.dot(NOW_TEMPS) ## 올해 평균기온 벡터와 과거 평균기온 벡터 사이의 내적이다.
                    
                    ## 두 벡터 사이의 각을 코사인 역함수를 이용하여 구했다.
                    angle = np.arccos(dot_product/(magnitudeNow*magnitudePast))
                    
                    ## 이렇게 구한 각차이를 딕셔너리에 저장해준다.
                    COSINE[year] = angle
                
                ## 딕셔너리를 튜플 리스트형태로 바꾼 후, 차이를 기준으로 오름차순 정렬해준다.
                COSINE = sorted(COSINE.items(), key=lambda x : x[1])

                ## 정렬된 DIFF 배열의 가장 앞 세개의 자료가 올해 년도와 가장 유사한 자료들이다.
                top1 = COSINE[0]
                top2 = COSINE[1]
                top3 = COSINE[2]


            ## 3. My Similarity
            ## --> 올해 평균기온들과 과거 평균기온들의 차이의 제곱을 모두 합한 것을 이용하여, 이 값이 작으면 작을수록 유사도가 크다고 생각한다.
            ## --> 날씨는 카오스이론에 따르므로, 미세한 차이에도 민감하다. 이 차이의 효과를 더 부각시키기 위한 장치로 제곱을 활용하였다.
            elif command == 3:
                
                ## 올해 평균기온 정보와 과거 평균기온 정보의 차이의 제곱을 저장하는 딕셔너리이다.
                DIFF = {}
                
                ## 과거의 정보들을 모두 돌며
                for year, temps in PAST_TEMPS_DICT.items():

                    ## 올해의 평균기온 정보와 과거의 평균기온 정보의 차이의 제곱의 합을 저장한 후
                    ## DIFF 딕셔너리에 저장해준다.
                    diff = ((NOW_TEMPS - temps)**2).sum()
                    DIFF[year] = diff

                ## 딕셔너리를 튜플 리스트형태로 바꾼 후, 차이를 기준으로 오름차순 정렬해준다.
                DIFF = sorted(DIFF.items(), key=lambda x : x[1])
                
                # 정렬된 DIFF 배열의 가장 앞 세개의 자료가 올해 년도와 가장 유사한 자료들이다.
                top1 = DIFF[0]
                top2 = DIFF[1]
                top3 = DIFF[2]
                
            
            ## 4. 종료
            elif command == 4:
                break
                
            
            ## 5. 이외의 정수가 입력되도 다시 입력하기
            else:
                print("1~4 사이의 정수를 입력하세요.\n\n")
                continue




            topTemps = [] ## 세 개의 년도의 내일 날짜의 평균기온을 저장하는 리스트이다. 크기는 3이다.
            nowMonth = now.month
            nowDay = now.day + 1
            
            ## 가장 유사한 세 개의 년도에 대해
            for nowYear in (top1[0], top2[0], top3[0]):

                ## 크롤링을 할 URL을 문자열로 저장한다.
                URL = "https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy="+str(nowYear)+"&mm="+str(nowMonth)+"&obs=1"

                ## HTTP GET Request
                req = requests.get(URL)

                ## HTML 소스 문자열 형식으로 가져온다.
                html = req.text

                ## BeautifulSoup으로 문자열 형식의 html소스를 python객체로 변환한다.
                ## 첫 인자는 html소스코드, 두 번째 인자는 어떤 parser를 이용할지 명시한다.
                ## 이번 프로젝트에서는 Python 내장 html.parser를 이용한다.
                soup = BeautifulSoup(html, 'html.parser')


                ## weathers에 td 태그 안의 span태그로 둘러쌓인 정보들만 뽑아서 저장한다.
                weathers = soup.select("td > span")


                ## avgTemp : 평균기온을 뽑아내는 정규표현식
                avgTemp = re.compile(r"(?<=평균기온:).*(?=℃)")


                ## 며칠째인지 체크하기
                dayCount = 1

                ## td태그안의 span태그로 둘러쌓인 모든 정보들을 탐색한다.
                for weather in weathers:

                    ## avgTempSearch : avgTemp 정규식으로 평균기온을 찾는다.
                    avgTempSearch = avgTemp.search(weather.text)

                    ## 평균기온이 찾아졌다면, nowAvgTemp에 평균기온을 저장해준다.
                    if avgTempSearch:
                        
                        ## 아직 내일 날짜를 찾지 못했다면, 다음 날짜를 탐색하러 간다.
                        if dayCount < nowDay:
                            dayCount += 1

                        ## 내일 날짜를 찾았다면, 그 날짜에 해당하는 평균기온 정보를 topTemps에 추가해준다.
                        elif dayCount == nowDay:
                            topTemps.append(float(avgTempSearch.group()))
                            break


            print()
            print()
            print(f"1번째로 유사한 년도: {top1[0]}")
            print(f"2번째로 유사한 년도: {top2[0]}")
            print(f"3번째로 유사한 년도: {top3[0]}")
            
            ## 내일 날씨를 세 개의 년도의 내일 날짜의 날씨를 유사도를 적용하여 가중평균을 구하였다.
            expected_temp = ((topTemps[0]* 1/top1[1]) + (topTemps[1]* 1/top2[1]) + (topTemps[2]* 1/top3[1])) / (1/top1[1] + 1/top2[1] + 1/top3[1])

            now = datetime.now() + timedelta(days=1)
            print(f"{now.year}년 {now.month}월 {now.day}일의 예상 기온: {expected_temp:.2f}\n")
            print("오늘의 옷차림은..")
            UTIL.weatherClothes(expected_temp)
            print()


        print(f"\n\n------------------------------------------------------- 내일의 옷차림 추천 종료 -------------------------------------------------------")

In [ ]:
weather = Weather()